In [ ]:
library(xml2)

In [ ]:
ComXmlFile  = read_xml("data/com.xml")
ContratXmlFile = read_xml("data/contract.xml")
EngiXmlFile = read_xml("data/engi.xml")

In [ ]:
# Chargement des bibliothèques nécessaires
library(xml2)     # Pour lire et manipuler des fichiers XML
library(dplyr)    # Pour manipuler des tableaux de données (tibbles)
library(purrr)    # Pour la fonctions de programmation fonctionnelle  (walk)
library(stringr)  # Pour manipuler les chaînes de caractères
library(tibble)

# Fonction principale : transforme un document XML en une liste de tables par classe
xmlToTables <- function(doc){
  tryCatch({
    # Vérification préliminaire
    if (!inherits(doc, "xml_document")) {
      stop("L'entrée doit être un objet xml_document")
    }
    
    root <- xml_root(doc)
    if (is.null(root)) {
      warning("Le document XML est vide")
      return(list())
    }

    tables <- list()

    parse_obj <- function(node){
      tryCatch({
        clas <- xml_attr(node, "clas")
        if (is.na(clas) || is.null(clas)) return()

        # Sécurisation de l'extraction des propriétés
        props <- xml_find_all(node, "./prop[not(*) and not(.//obj)]")
        if (length(props) == 0) return()

        # Récupération sécurisée des textes et noms
        texts <- xml_text(props) %>% trimws()
        names <- xml_attr(props, "nom") %>% str_remove("^\\$")
        
        # Vérification de la cohérence des longueurs
        if (length(texts) != length(names)) {
          warning(sprintf("Incohérence détectée dans %s : %d valeurs pour %d noms", 
                         clas, length(texts), length(names)))
          return()
        }

        row <- set_names(texts, names)

        # Ajout sécurisé à la table
        if (is.null(tables[[clas]])) {
          tables[[clas]] <<- tibble()
        }
        
        # Conversion sécurisée en tibble
        safe_row <- tryCatch({
          as_tibble_row(row)
        }, error = function(e) {
          warning(sprintf("Erreur lors de la conversion en tibble pour %s : %s", 
                         clas, e$message))
          return(NULL)
        })
        
        if (!is.null(safe_row)) {
          tables[[clas]] <<- tryCatch({
            bind_rows(tables[[clas]], safe_row)
          }, error = function(e) {
            warning(sprintf("Erreur lors du bind_rows pour %s : %s", 
                           clas, e$message))
            tables[[clas]]
          })
        }

        # Traitement sécurisé des enfants
        children <- tryCatch({
          c(xml_find_all(node, "./prop//obj"),
            xml_find_all(node, "./prop//entr/obj"))
        }, error = function(e) {
          warning(sprintf("Erreur lors de la recherche des enfants : %s", e$message))
          xml_nodeset()
        })
        
        walk(children, parse_obj)

      }, error = function(e) {
        warning(sprintf("Erreur lors du traitement du nœud : %s", e$message))
      })
    }

    # Traitement principal sécurisé
    objs <- tryCatch({
      xml_find_all(root, ".//obj")
    }, error = function(e) {
      warning(sprintf("Erreur lors de la recherche des objets : %s", e$message))
      xml_nodeset()
    })
    
    if (length(objs) > 0) {
      walk(objs, parse_obj)
    } else {
      warning("Aucun objet <obj> trouvé dans le document")
    }

    # Nettoyage sécurisé des noms de colonnes
    tables <- tryCatch({
      map(tables, function(tbl) {
        if (nrow(tbl) > 0) {
          tryCatch({
            rename_with(tbl, ~str_replace_all(.x, "[^[:alnum:]_]", ""))
          }, error = function(e) {
            warning(sprintf("Erreur lors du nettoyage des noms : %s", e$message))
            tbl
          })
        } else {
          tbl
        }
      })
    }, error = function(e) {
      warning(sprintf("Erreur lors du nettoyage final : %s", e$message))
      tables
    })

    return(tables)

  }, error = function(e) {
    warning(sprintf("Erreur globale dans xmlToTables : %s", e$message))
    return(list())
  })
}


In [ ]:
engi_df = xmlToTables(ContratXmlFile)
engi_df

In [ ]:
xmlToTables2 <- function(doc, exclude_clas = NULL) {
  tryCatch({
    # Vérification préliminaire
    if (!inherits(doc, "xml_document")) {
      stop("L'entrée doit être un objet xml_document")
    }
    
    root <- xml_root(doc)
    if (is.null(root)) {
      warning("Le document XML est vide")
      return(list())
    }

    # Normalisation de exclude_clas en vecteur de caractères
    if (is.null(exclude_clas)) {
      exclude_clas <- character(0)
    } else {
      exclude_clas <- as.character(exclude_clas)
    }

    tables <- list()

    parse_obj <- function(node) {
      tryCatch({
        clas <- xml_attr(node, "clas")
        if (is.na(clas) || is.null(clas) || str_to_lower(clas) %in% exclude_clas) return()

        # Sécurisation de l'extraction des propriétés
        props <- xml_find_all(node, "./prop[not(*) and not(.//obj)]")
        if (length(props) == 0) return()

        # Récupération sécurisée des textes et noms
        texts <- xml_text(props) %>% trimws()
        names <- xml_attr(props, "nom") %>% str_remove("^\\$")
        
        # Vérification de la cohérence des longueurs
        if (length(texts) != length(names)) {
          warning(sprintf("Incohérence détectée dans %s : %d valeurs pour %d noms", 
                         clas, length(texts), length(names)))
          return()
        }

        row <- set_names(texts, names)

        # Ajout sécurisé à la table
        if (is.null(tables[[clas]])) {
          tables[[clas]] <<- tibble()
        }
        
        # Conversion sécurisée en tibble
        safe_row <- tryCatch({
          as_tibble_row(row)
        }, error = function(e) {
          warning(sprintf("Erreur lors de la conversion en tibble pour %s : %s", 
                         clas, e$message))
          return(NULL)
        })
        
        if (!is.null(safe_row)) {
          tables[[clas]] <<- tryCatch({
            bind_rows(tables[[clas]], safe_row)
          }, error = function(e) {
            warning(sprintf("Erreur lors du bind_rows pour %s : %s", 
                           clas, e$message))
            tables[[clas]]
          })
        }

        # Traitement sécurisé des enfants
        children <- tryCatch({
          c(xml_find_all(node, "./prop//obj"),
            xml_find_all(node, "./prop//entr/obj"))
        }, error = function(e) {
          warning(sprintf("Erreur lors de la recherche des enfants : %s", e$message))
          xml_nodeset()
        })
        
        walk(children, parse_obj)

      }, error = function(e) {
        warning(sprintf("Erreur lors du traitement du nœud : %s", e$message))
      })
    }

    # Traitement principal sécurisé
    objs <- tryCatch({
      xml_find_all(root, ".//obj")
    }, error = function(e) {
      warning(sprintf("Erreur lors de la recherche des objets : %s", e$message))
      xml_nodeset()
    })
    
    if (length(objs) > 0) {
      walk(objs, parse_obj)
    } else {
      warning("Aucun objet <obj> trouvé dans le document")
    }

    # Nettoyage sécurisé des noms de colonnes
    tables <- tryCatch({
      map(tables, function(tbl) {
        if (nrow(tbl) > 0) {
          tryCatch({
            rename_with(tbl, ~str_replace_all(.x, "[^[:alnum:]_]", ""))
          }, error = function(e) {
            warning(sprintf("Erreur lors du nettoyage des noms : %s", e$message))
            tbl
          })
        } else {
          tbl
        }
      })
    }, error = function(e) {
      warning(sprintf("Erreur lors du nettoyage final : %s", e$message))
      tables
    })

    return(tables)

  }, error = function(e) {
    warning(sprintf("Erreur globale dans xmlToTables : %s", e$message))
    return(list())
  })
}


In [ ]:
engi_df = xmlToTables2(ContratXmlFile,exclude_clas = c("etape","contrat"))
engi_df

In [321]:
xmlToTables_exclu <- function(doc,
                        exclude_clas_containing = NULL) {
  tryCatch({
    # --- vérifications initiales ---
    if (!inherits(doc, "xml_document")) {
      stop("L'entrée doit être un objet xml_document")
    }
    
    root <- xml_root(doc)
    if (is.null(root)) {
      warning("Le document XML est vide")
      return(list())
    }

    # --- préparation des motifs d'exclusion ---
    exclude_patterns <- as.character(exclude_clas_containing %||% character(0))
    has_exclusions <- length(exclude_patterns) > 0

    tables <- list()

    parse_obj <- function(node) {
      tryCatch({
        clas <- xml_attr(node, "clas")
        if (is.na(clas) || is.null(clas)) return()

        # Exclusion si le nom de classe contient un motif indésirable
        if (has_exclusions) {
          drop <- any(str_detect(clas, regex(exclude_patterns, ignore_case = TRUE)))
          if (drop) return()
        }

        props <- xml_find_all(node, "./prop[not(*) and not(.//obj)]")
        if (length(props) == 0) return()

        texts <- xml_text(props) %>% trimws()
        names <- xml_attr(props, "nom") %>% str_remove("^\\$")
        
        if (length(texts) != length(names)) {
          warning(sprintf("Incohérence dans %s : %d valeurs / %d noms",
                          clas, length(texts), length(names)))
          return()
        }

        row <- set_names(texts, names)

        if (is.null(tables[[clas]])) tables[[clas]] <<- tibble()
        tables[[clas]] <<- tryCatch(
          bind_rows(tables[[clas]], as_tibble_row(row)),
          error = function(e) {
            warning(sprintf("bind_rows échoue pour %s : %s", clas, e$message))
            tables[[clas]]
          }
        )

        children <- c(xml_find_all(node, "./prop//obj"),
                      xml_find_all(node, "./prop//entr/obj"))
        walk(children, parse_obj)

      }, error = function(e) {
        warning(sprintf("Erreur lors du traitement du nœud : %s", e$message))
      })
    }

    objs <- tryCatch(xml_find_all(root, ".//obj"),
                     error = function(e) xml_nodeset())
    if (length(objs)) walk(objs, parse_obj)

    tables <- map(tables, ~ tryCatch(
      rename_with(., ~ str_replace_all(.x, "[^[:alnum:]_]", "")),
      error = function(e) .
    ))

    return(tables)

  }, error = function(e) {
    warning("Erreur globale :", e$message)
    return(list())
  })
}

In [ ]:
engi_df = xmlToTables_exclu(EngiXmlFile,exclude_clas = c("lay","item",'seg','zone'))
print(engi_df)
